In [1]:
from collections import defaultdict
from collections import deque
import re
from collections import Counter
import itertools

from string import ascii_lowercase,ascii_uppercase

In [32]:

def bfs(connections, start, goal=None):
    """
    Requires a dict with neighbors per node.
    Or a function returning neighbors per node

    Returns
    if goal == None:    return all paths from start
    elif goal found:    returns path to goal
    else:               returns False
    """
    seen = set() # the locations that have been explored
    frontier = deque([start]) # the locations that still need to be visited
    # paths = {start: [start]}
    isfunction = callable(connections)
    parents = {start.__repr__(): None}

    def get_path(parents):
        # print(start,goals)
        cur = goal.__repr__()
        path = [cur]
        while cur != start.__repr__():
            cur = parents[cur]
            path.append(cur)
        path.reverse()
        return path
    count = 0
    print(parents)
    while frontier:
        count+=1
        if count%1000 == 0:
            print(len(frontier),len(parents),len(seen))
            print(frontier[0])
        search = frontier.popleft()
        if isfunction: neighbors = connections(search)
        else: neighbors = connections.get(search,None)
        if neighbors:
            for n in neighbors:
                if n.__repr__() not in seen:
                    seen.add(n.__repr__())
                    frontier.append(n)
                    parents[n.__repr__()]= search.__repr__()
                    if goal and n == goal:
                        print('goal found')
                        return get_path(parents)
                        # return paths[goal],parents
        seen.add(search.__repr__())
    print('done')
    if goal: return False
    else: return parents

class State(dict):
    def __init__(self,d):
        super().__init__(d)
        self.set_available()

    def __repr__(self):
        res = 'floor '+' floor '.join([v if v else 'none' for k,v in self.items()])
        return res

    def num_loose(self):
        res = ''.join([v if v else '' for k,v in self.items()])
        return int(len(re.findall('[a-zA-Z]',res))/2)

    def num_items(self):
        count = 0
        for i in range(4):
            count += int(self[i][0])*2
            count += len(self[i])-1
        if count == 14: return True
        else: return False

    def set_available(self):
        avail = []
        res = ''.join([v if v else '' for k,v in self.items() if k in [0,1,2,3]])
        self.available = ''.join(sorted(set('abcdefghi')-set(res)))


    def recode(self):
        res = ''.join([v for k,v in self.items() if k in [0,1,2,3] and v])
        res = ''.join([c for c in res if c.isalpha() and c.islower()])
        code = ''.maketrans(res+res.upper(), ascii_lowercase[:len(res)]+ascii_uppercase[:len(res)])
        for k in self:
            if k in [0,1,2,3]:
                super().__setitem__(k,self[k].translate(code))
                self.set_available()

    def get_next_two_letters(self):
        return self.available[0:2]

    def __setitem__(self,k,v):
        if (k != 'elev' and v):
            v = ''.join(sorted(v))
            num_pairs = int(v[0])
            to_remove = [key for key,val in Counter(v.lower()).items() if val > 1]
            for i in to_remove: self.available+i
            num_pairs+=len(to_remove)
            newstring = str(num_pairs)
            for i in v:
                if not i.lower() in to_remove and not i.isdigit():
                    newstring+=i
            super().__setitem__(k,newstring)
        else:
            super().__setitem__(k,v)

state = {0:'4', 1: '2A',2:'0a',3:'0','elev':'0'}
goal = {0:'0',1:'0',2:'0',3:'7','elev':'3'}

def isvalid(inp):
    for i in inp:
        if i.isalpha() and i.islower() and i.upper() not in inp:
            for j in inp:
                if (j.isdigit() and int(j)>0) or (j.isupper() and j.lower() != i): return False
    return True
nextfloor = {0:[1],1:[0,2],2:[1,3],3:[2]}

def getneigh(state):
    newstates = []
    cur_floor = int(state['elev'])
    next_two_letters = state.get_next_two_letters()
    current_floor = state[cur_floor] # used to be copy
    if int(current_floor[0])>0:
        current_floor+=next_two_letters[0].upper()+next_two_letters[0].lower()
        current_floor = str(int(current_floor[0])-1)+current_floor[1:]
    if int(current_floor[0])>0:
        current_floor+=next_two_letters[1].upper()+next_two_letters[1].lower()
        current_floor = str(int(current_floor[0])-1)+current_floor[1:]

    for comb in itertools.chain((itertools.combinations(current_floor[1:],2)), current_floor[1:]):
        comb = ''.join(comb)
        new_floor = ''.join([i for i in current_floor if i not in comb])
        if isvalid(comb) and isvalid(new_floor): # current floor is ok
            for n in nextfloor[cur_floor]:
                if isvalid(state[n] + comb): # new floor ok
                    newstate = State(state.copy())
                    newstate['elev']=str(n)
                    newstate[cur_floor] = new_floor
                    newstate[n] = newstate[n] + comb
                    newstate.recode()
                    assert newstate.num_items()
                    newstates.append(newstate)
    return newstates

In [18]:
50ms

SyntaxError: invalid syntax (<ipython-input-18-1c776871a302>, line 1)

In [34]:
a = State(state)
%timeit for i in range(100): getneigh(a)

52.1 ms ± 5.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
a = State(state)
# %timeit for i in range(10): getneigh(a)
%load_ext line_profiler
%lprun -f getneigh getneigh(a)
# res = bfs(getneigh,State(state),State(goal))
# len(res)



The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Timer unit: 1e-07 s

Total time: 0.0014457 s
File: <ipython-input-22-6c6d3ff12c60>
Function: getneigh at line 116

Line #      Hits         Time  Per Hit   % Time  Line Contents
   116                                           def getneigh(state):
   117         1         57.0     57.0      0.4      newstates = []
   118         1         92.0     92.0      0.6      cur_floor = int(state['elev'])
   119         1        105.0    105.0      0.7      next_two_letters = state.get_next_two_letters()
   120         1         34.0     34.0      0.2      current_floor = state[cur_floor] # used to be copy
   121         1         44.0     44.0      0.3      if int(current_floor[0])>0:
   122         1         69.0     69.0      0.5          current_floor+=next_two_letters[0].upper()+next_two_letters[0].lower()
   123         1         87.0     87.0      0.6          current_floor = str(int(current_floor[0])-1)+current_floor[1:]
   124         1         37.0     37.0      0.3      if int(curren

In [25]:
%lprun -f a.recode a.recode()

Timer unit: 1e-07 s

Total time: 0.0003259 s
File: <ipython-input-22-6c6d3ff12c60>
Function: recode at line 78

Line #      Hits         Time  Per Hit   % Time  Line Contents
    78                                               def recode(self):
    79         1        387.0    387.0     11.9          res = ''.join([v for k,v in self.items() if k in [0,1,2,3] and v])
    80         1        309.0    309.0      9.5          res = ''.join([c for c in res if c.isalpha() and c.islower()])
    81         1        221.0    221.0      6.8          code = ''.maketrans(res+res.upper(), ascii_lowercase[:len(res)]+ascii_uppercase[:len(res)])
    82         6        212.0     35.3      6.5          for k in self:
    83         5        160.0     32.0      4.9              if k in [0,1,2,3]:
    84         4        464.0    116.0     14.2                  super().__setitem__(k,self[k].translate(code))
    85         4       1506.0    376.5     46.2                  self.set_available()

In [26]:
%lprun -f a.num_items a.num_items()

Timer unit: 1e-07 s

Total time: 5.5e-05 s
File: <ipython-input-22-6c6d3ff12c60>
Function: num_items at line 64

Line #      Hits         Time  Per Hit   % Time  Line Contents
    64                                               def num_items(self):
    65         1         49.0     49.0      8.9          count = 0
    66         5        134.0     26.8     24.4          for i in range(4):
    67         4        231.0     57.8     42.0              count += int(self[i][0])*2
    68         4        113.0     28.2     20.5              count += len(self[i])-1
    69         1         23.0     23.0      4.2          if count == 14: return True
    70                                                   else: return False

In [27]:
%lprun -f a.__setitem__ a.__setitem__(0,"1AbB")

Timer unit: 1e-07 s

Total time: 0.0001579 s
File: <ipython-input-22-6c6d3ff12c60>
Function: __setitem__ at line 90

Line #      Hits         Time  Per Hit   % Time  Line Contents
    90                                               def __setitem__(self,k,v):
    91         1         80.0     80.0      5.1          if (k != 'elev' and v):
    92         1        140.0    140.0      8.9              v = ''.join(sorted(v))
    93         1         92.0     92.0      5.8              num_pairs = int(v[0])
    94         1        742.0    742.0     47.0              to_remove = [key for key,val in Counter(v.lower()).items() if val > 1]
    95         2         74.0     37.0      4.7              for i in to_remove: self.available+i
    96         1         33.0     33.0      2.1              num_pairs+=len(to_remove)
    97         1         51.0     51.0      3.2              newstring = str(num_pairs)
    98         5        116.0     23.2      7.3              for i in v:
    99        

In [28]:
%lprun -f isvalid isvalid("1AaBbc")

Timer unit: 1e-07 s

Total time: 8.57e-05 s
File: <ipython-input-22-6c6d3ff12c60>
Function: isvalid at line 108

Line #      Hits         Time  Per Hit   % Time  Line Contents
   108                                           def isvalid(inp):
   109         6        263.0     43.8     30.7      for i in inp:
   110         6        414.0     69.0     48.3          if i.isalpha() and i.islower() and i.upper() not in inp:
   111         1         39.0     39.0      4.6              for j in inp:
   112         1        141.0    141.0     16.5                  if (j.isdigit() and int(j)>0) or (j.isupper() and j.lower() != i): return False
   113                                               return True

In [29]:
%lprun -f a.set_available a.set_available()

Timer unit: 1e-07 s

Total time: 4.22e-05 s
File: <ipython-input-22-6c6d3ff12c60>
Function: set_available at line 72

Line #      Hits         Time  Per Hit   % Time  Line Contents
    72                                               def set_available(self):
    73         1         39.0     39.0      9.2          avail = []
    74         1        172.0    172.0     40.8          res = ''.join([v if v else '' for k,v in self.items() if k in [0,1,2,3]])
    75         1        211.0    211.0     50.0          self.available = ''.join(sorted(set('abcdefghi')-set(res)))